In [117]:
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
import re 
import pickle
import gensim, logging
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
from gensim.models import CoherenceModel, ldamodel
import psycopg2


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/marieskoczylas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/marieskoczylas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/marieskoczylas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [118]:
conn = psycopg2.connect(host = "localhost", dbname = "rolemodel", user = "marieskoczylas", port=5432)
cur = conn.cursor()
conn

<connection object at 0x1a251722d8; dsn: 'host=localhost dbname=rolemodel user=marieskoczylas port=5432', closed: 0>

In [119]:
cur.execute("SELECT profile_id, profile FROM profiles")
biglist = cur.fetchall()
print(biglist)

[(998, 'Berenice Abbott (July 17, 1898 – December 9, 1991), née Bernice Alice Abbott, was an American photographer best known for her portraits of between-the-wars 20th-century cultural figures, New York City photographs of architecture and urban design of the 1930s, and science interpretation in the 1940s–1960s.\n\nEarly years\nAbbott was born in Springfield, Ohio, the youngest of four children. and brought up there by her divorced mother, née Lillian Alice Bunn (m. Charles E. Abbott in Chillicothe OH, 1886).\nShe took college preparatory classes and graduated in 1917. She attended Ohio State University for two semesters, but left in early 1918 because her German professor was dismissed because he was a German teaching an English class. Abbot wanted to take a job in journalism, but later lost interest in it and followed the path of photography because of the interaction with Eugene O’Neill after that she became attached to photography . Abbott was also a well known photographer in Par

In [13]:
# Create the sample list (of tuples)

traintwotest = []

for woman in biglist:
    woman_id = woman[0]
    if woman_id in [1059, 1065]:
        traintwotest.append(woman) 

print(traintwotest)

[(1059, 'Jamie Lee Haden-Guest, Baroness Haden-Guest (née Curtis; born November 22, 1958), commonly known as Jamie Lee Curtis, is an American actress and author. She made her film debut in 1978 by starring as Laurie Strode in John Carpenter\'s Halloween (1978). A big hit, the film established her as a notable actress in horror, and she subsequently starred in Halloween II (1981), The Fog (1980), Prom Night (1980), Terror Train (1980), and Roadgames (1981), gaining the status of "scream queen" to mainstream audiences. Curtis has since compiled a body of work that spans many film genres, including the cult comedy films Trading Places (1983), for which she won a BAFTA Award for Best Actress in a Supporting Role, A Fish Called Wanda (1988), and True Lies (1994), for which she won a Golden Globe Award for Best Actress – Motion Picture Musical or Comedy.\nCurtis\'s other notable films include Blue Steel (1989) and Freaky Friday (2003), earning a third Golden Globe nomination for Best Actress

In [14]:
# Lowercase the profile text

traintwotestlower = []

for profiletuple in traintwotest:
    traintwotestlower.append((profiletuple[0], profiletuple[1].lower()))

print(traintwotestlower)

[(1059, 'jamie lee haden-guest, baroness haden-guest (née curtis; born november 22, 1958), commonly known as jamie lee curtis, is an american actress and author. she made her film debut in 1978 by starring as laurie strode in john carpenter\'s halloween (1978). a big hit, the film established her as a notable actress in horror, and she subsequently starred in halloween ii (1981), the fog (1980), prom night (1980), terror train (1980), and roadgames (1981), gaining the status of "scream queen" to mainstream audiences. curtis has since compiled a body of work that spans many film genres, including the cult comedy films trading places (1983), for which she won a bafta award for best actress in a supporting role, a fish called wanda (1988), and true lies (1994), for which she won a golden globe award for best actress – motion picture musical or comedy.\ncurtis\'s other notable films include blue steel (1989) and freaky friday (2003), earning a third golden globe nomination for best actress

In [18]:
# Tokenize and leave out punctuation

from nltk.tokenize import RegexpTokenizer
re_tokenizer = RegexpTokenizer(r'\w+')

traintwotesttokens = []

for profiletuple in traintwotestlower:
    traintwotesttokens.append((profiletuple[0], re_tokenizer.tokenize(profiletuple[1])))

print(traintwotesttokens)

[(1059, ['jamie', 'lee', 'haden', 'guest', 'baroness', 'haden', 'guest', 'née', 'curtis', 'born', 'november', '22', '1958', 'commonly', 'known', 'as', 'jamie', 'lee', 'curtis', 'is', 'an', 'american', 'actress', 'and', 'author', 'she', 'made', 'her', 'film', 'debut', 'in', '1978', 'by', 'starring', 'as', 'laurie', 'strode', 'in', 'john', 'carpenter', 's', 'halloween', '1978', 'a', 'big', 'hit', 'the', 'film', 'established', 'her', 'as', 'a', 'notable', 'actress', 'in', 'horror', 'and', 'she', 'subsequently', 'starred', 'in', 'halloween', 'ii', '1981', 'the', 'fog', '1980', 'prom', 'night', '1980', 'terror', 'train', '1980', 'and', 'roadgames', '1981', 'gaining', 'the', 'status', 'of', 'scream', 'queen', 'to', 'mainstream', 'audiences', 'curtis', 'has', 'since', 'compiled', 'a', 'body', 'of', 'work', 'that', 'spans', 'many', 'film', 'genres', 'including', 'the', 'cult', 'comedy', 'films', 'trading', 'places', '1983', 'for', 'which', 'she', 'won', 'a', 'bafta', 'award', 'for', 'best', 'a

In [110]:
# Stopword removal

stop_words = set(stopwords.words('english'))

def remove_stopwords(texts):
    output = []
    for word in texts:
        processed = simple_preprocess(str(word))
        # removes "empty" processed words and stopwords
        if len(processed) > 0 and processed[0] not in stop_words:  
            output.append(processed)            
    return output
            
            
#        some_list = []
#        for word in simple_preprocess(str(doc)):
#            if word not in stop_words
        
    #return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

train2tokensclean = []

for profiletuple in traintwotesttokens:
    train2tokensclean.append((profiletuple[0], remove_stopwords(profiletuple[1])))

print(train2tokensclean)

#AND ADDING STOPWORDS (for later)
#stopwords = nltk.corpus.stopwords.words('english')
#newStopWords = ['stopWord1','stopWord2']
#stopwords.extend(newStopWords)


[(1059, [['jamie'], ['lee'], ['haden'], ['guest'], ['baroness'], ['haden'], ['guest'], ['née'], ['curtis'], ['born'], ['november'], ['commonly'], ['known'], ['jamie'], ['lee'], ['curtis'], ['american'], ['actress'], ['author'], ['made'], ['film'], ['debut'], ['starring'], ['laurie'], ['strode'], ['john'], ['carpenter'], ['halloween'], ['big'], ['hit'], ['film'], ['established'], ['notable'], ['actress'], ['horror'], ['subsequently'], ['starred'], ['halloween'], ['ii'], ['fog'], ['prom'], ['night'], ['terror'], ['train'], ['roadgames'], ['gaining'], ['status'], ['scream'], ['queen'], ['mainstream'], ['audiences'], ['curtis'], ['since'], ['compiled'], ['body'], ['work'], ['spans'], ['many'], ['film'], ['genres'], ['including'], ['cult'], ['comedy'], ['films'], ['trading'], ['places'], ['bafta'], ['award'], ['best'], ['actress'], ['supporting'], ['role'], ['fish'], ['called'], ['wanda'], ['true'], ['lies'], ['golden'], ['globe'], ['award'], ['best'], ['actress'], ['motion'], ['picture'], 

In [31]:
# In the future I could do some lemmatization to reduce differential forms of a word to a common base form.
# Lemmitization is a more sophisticated process than stemming - it can account for variables such as part-of-speech, meaning, and context within a document or neighboring sentences.
# But I'd need to fix because lemmatize expects a string not a list of strings.

#lemmatizer = WordNetLemmatizer()
#train2lems = []

#for profiletuple in train2tokensclean:
#    train2lems.append((profiletuple[0], lemmatizer.lemmatize(profiletuple[1])))

#print(train2lems)

TypeError: unhashable type: 'list'

In [113]:
# Finally, vectorization: representing the text as a quantitative set of features for subsequent analysis.


# Input: cleanedwords [(id, [[tokens]])]
# Output: [(id, [(word id, word frequency)], id2word)]
def vector_this(cleanedwords):
    train2vec = []
    for tokentuple in cleanedwords:
        tokens = tokentuple[1]
        id2wordtest = corpora.Dictionary(tokens)
        corpora_id = [id2wordtest.doc2bow(token) for token in tokens]
        train2vec.append((tokentuple[0], corpora_id, id2wordtest))
    return train2vec

traincorpus = vector_this(train2tokensclean)
print(traincorpus)


[(1059, [[(0, 1)], [(1, 1)], [(2, 1)], [(3, 1)], [(4, 1)], [(2, 1)], [(3, 1)], [(5, 1)], [(6, 1)], [(7, 1)], [(8, 1)], [(9, 1)], [(10, 1)], [(0, 1)], [(1, 1)], [(6, 1)], [(11, 1)], [(12, 1)], [(13, 1)], [(14, 1)], [(15, 1)], [(16, 1)], [(17, 1)], [(18, 1)], [(19, 1)], [(20, 1)], [(21, 1)], [(22, 1)], [(23, 1)], [(24, 1)], [(15, 1)], [(25, 1)], [(26, 1)], [(12, 1)], [(27, 1)], [(28, 1)], [(29, 1)], [(22, 1)], [(30, 1)], [(31, 1)], [(32, 1)], [(33, 1)], [(34, 1)], [(35, 1)], [(36, 1)], [(37, 1)], [(38, 1)], [(39, 1)], [(40, 1)], [(41, 1)], [(42, 1)], [(6, 1)], [(43, 1)], [(44, 1)], [(45, 1)], [(46, 1)], [(47, 1)], [(48, 1)], [(15, 1)], [(49, 1)], [(50, 1)], [(51, 1)], [(52, 1)], [(53, 1)], [(54, 1)], [(55, 1)], [(56, 1)], [(57, 1)], [(58, 1)], [(12, 1)], [(59, 1)], [(60, 1)], [(61, 1)], [(62, 1)], [(63, 1)], [(64, 1)], [(65, 1)], [(66, 1)], [(67, 1)], [(57, 1)], [(58, 1)], [(12, 1)], [(68, 1)], [(69, 1)], [(70, 1)], [(52, 1)], [(6, 1)], [(26, 1)], [(53, 1)], [(71, 1)], [(72, 1)], [(73, 1

In [115]:
# LDA Model

lda_model = gensim.models.ldamodel.LdaModel(corpus=traincorpus[0][1],
                                           id2word=traincorpus[0][2],
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

print(lda_model.print_topics())


[(0, '0.198*"film" + 0.040*"globe" + 0.025*"tap" + 0.025*"every" + 0.025*"marry" + 0.025*"never" + 0.025*"oh" + 0.025*"using" + 0.025*"adopted" + 0.025*"addicted"'), (1, '0.050*"june" + 0.038*"hollywood" + 0.036*"tv" + 0.033*"live" + 0.032*"blood" + 0.032*"got" + 0.032*"kael" + 0.032*"together" + 0.032*"news" + 0.032*"son"'), (2, '0.067*"role" + 0.065*"warcraft" + 0.045*"website" + 0.039*"later" + 0.036*"starred" + 0.033*"cosplay" + 0.033*"achievement" + 0.033*"chance" + 0.033*"commentary" + 0.033*"fan"'), (3, '0.076*"december" + 0.029*"routine" + 0.029*"serving" + 0.029*"becoming" + 0.029*"couple" + 0.029*"watched" + 0.023*"past" + 0.022*"huffington" + 0.018*"wilshire" + 0.018*"causes"'), (4, '0.042*"campaign" + 0.042*"women" + 0.031*"based" + 0.023*"harmon" + 0.021*"reviews" + 0.021*"victims" + 0.021*"stories" + 0.021*"twelve" + 0.021*"offering" + 0.021*"hopkins"'), (5, '0.110*"guest" + 0.054*"laura" + 0.049*"los" + 0.037*"little" + 0.029*"two" + 0.023*"subsequently" + 0.019*"school"

In [116]:
for corpus in traincorpus:
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus[1],
                                           id2word=corpus[2],
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
    print(lda_model.print_topics())

[(0, '0.198*"film" + 0.040*"globe" + 0.025*"tap" + 0.025*"every" + 0.025*"marry" + 0.025*"never" + 0.025*"oh" + 0.025*"using" + 0.025*"adopted" + 0.025*"addicted"'), (1, '0.050*"june" + 0.038*"hollywood" + 0.036*"tv" + 0.033*"live" + 0.032*"blood" + 0.032*"got" + 0.032*"kael" + 0.032*"together" + 0.032*"news" + 0.032*"son"'), (2, '0.067*"role" + 0.065*"warcraft" + 0.045*"website" + 0.039*"later" + 0.036*"starred" + 0.033*"cosplay" + 0.033*"achievement" + 0.033*"chance" + 0.033*"commentary" + 0.033*"fan"'), (3, '0.076*"december" + 0.029*"routine" + 0.029*"serving" + 0.029*"becoming" + 0.029*"couple" + 0.029*"watched" + 0.023*"past" + 0.022*"huffington" + 0.018*"wilshire" + 0.018*"causes"'), (4, '0.042*"campaign" + 0.042*"women" + 0.031*"based" + 0.023*"harmon" + 0.021*"reviews" + 0.021*"victims" + 0.021*"stories" + 0.021*"twelve" + 0.021*"offering" + 0.021*"hopkins"'), (5, '0.110*"guest" + 0.054*"laura" + 0.049*"los" + 0.037*"little" + 0.029*"two" + 0.023*"subsequently" + 0.019*"school"